# Core

> Core pieces needed to use during code generation.

In [ ]:
#| default_exp core

In [ ]:
#| export
from typing import Any
from copy import deepcopy
from textwrap import indent
from collections import ChainMap
from typing import Callable, Optional
from jinja2 import Environment, BaseLoader, Template, StrictUndefined
from frontmatter.default_handlers import YAMLHandler
from frontmatter.util import u

In [ ]:
#| hide
import nbdev
import nbdev.showdoc as showdoc
from fastcore.test import *
from jinja2 import UndefinedError

## Data

We want to build a code generator but before getting to that part, we need a class to hold some data for use while doing code generation. For that reason, here, we develop `Data`

First off, we will have a parent-children so we will represent that with a mixin. We don't directly put this into the main class because we'll need it for other usages later.

In [ ]:
#| export
class WithChildrenMixin:
    """Adds `parent`/`children`"""
    def __init__(self):
        self.parent = None
        self.children = []

    def __len__(self):
        return len(self.children)
    
    def __contains__(self, element):
        return element in self.children
            
    def add_child(self, child: "Data"):
        self.children.append(child)
        child.set_parent(self)
        return child
    
    def set_parent(self, parent: "Data"):
        self.parent = parent

So now, we can make the class to hold the data

In [ ]:
#| export
class Data(WithChildrenMixin):
    """Data holder used during code generation. Logic is kept as separate functions"""
    def __init__(self, 
                 name:str, # name of this element
                 attrs: dict[str, Any] = None): # attributes
        
        self.name = name
        
        if attrs is None:
            attrs = {}
        self._attrs = attrs
        
        super().__init__()
    
    @property
    def attrs(self):
        if self.parent:
            return ChainMap(self._attrs, self.parent.attrs)
        return ChainMap(self._attrs)
            
    def clone(self):
        return deepcopy(self)
        
    def __eq__(self, a):
        same_name = self.name == a.name
        same_attrs = self.attrs == a.attrs
        same_children = self.children == a.children
        return same_name and same_attrs and same_children
    
    def __str__(self):
        
        is_self_closing = not self.children
        
        if self.children:
            children = map(str, self.children)
            children = "\n".join(children)
            children = children.strip()
            children = f"\n{children}\n"
            children = indent(children, "    ")
            
        if self.attrs:
            if is_self_closing:
                return f"<{self.name} {dict(self.attrs)} />"
            else:
                return f"<{self.name} {dict(self.attrs)}>{children}</{self.name}>"
            
        if is_self_closing:
            return f"<{self.name} />"
        else:
            return f"<{self.name}>{children}</{self.name}>"
    
    __repr__ = __str__

In [ ]:
#| hide
showdoc.show_doc(Data)

---

### Data

>      Data (name:str, attrs:dict[str,typing.Any]=None)

Data holder used during code generation. Logic is kept as separate functions

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| name | str |  | name of this element |
| attrs | dict | None | attributes |

### Basic operations

Now that we have a class to hold our data, and only for now, we declare structures to use for code generation, manually. Later, we'll create some data-loaders.

In [ ]:
james = Data('person', {"name": "james"})

In [ ]:
james

<person {'name': 'james'} />

In [ ]:
#| hide
assert james.name == 'person'
assert james.attrs == {"name": "james"}

We can add children (note: a child's attributes will also include those of his parent)

In [ ]:
james = Data('person', {"name": "james"})
olive = james.add_child(Data('person', {"name": "olive"}))
silva = james.add_child(Data('person', {"name": "silva"}))

andrew = olive.add_child(Data('person', {"name": "andrew"}))

james.add_child(Data('person'))
olive.add_child(Data('person'))
olive.add_child(Data('person'))

<person {'name': 'olive'} />

and a child will know its parent

In [ ]:
olive.parent.attrs['name']

'james'

In [ ]:
#| hide
assert james == olive.parent
assert james.children[0].attrs["name"] == "olive"
assert silva.attrs['name'] == 'silva'
assert andrew.attrs["name"] == "andrew"

To check the number of children, simply use `len`

In [ ]:
len(james)

3

In [ ]:
#| hide
assert len(james) == 3

You can compare elements but they are tested based on their attributes and children

In [ ]:
b = Data('b', {"age": 22})
c = Data('b', {"age": 22})
d = Data('d')
b.add_child(d)
c.add_child(d)
b == c

True

In [ ]:
#| hide
b = Data('b')
c = Data('b')
assert b == c

b = Data('b')
c = Data('c')
assert b != c

b = Data('b', {"name": "santos"})
c = Data('b', {"name": "santos"})
assert b == c

b = Data('b', {"name": "silva"})
c = Data('b', {"name": "santos"})
assert b != c

b = Data('b', {"age": 22})
c = Data('b', {"age": 22})
d = Data('d')
b.add_child(d)
c.add_child(d)
assert b == c

You can test if an element is a child of another

In [ ]:
olive in james

True

In [ ]:
#| hide
assert olive in james

### Cloning

You can duplicate any `Data` instance

In [ ]:
james.clone()

<person {'name': 'james'}>
    <person {'name': 'olive'}>
        <person {'name': 'andrew'} />
        <person {'name': 'olive'} />
        <person {'name': 'olive'} />
    </person>
    <person {'name': 'silva'} />
    <person {'name': 'james'} />
</person>

In [ ]:
#| hide

root = Data('root')
root.add_child(Data('child'))

boot = root
assert root == boot

root.attrs["extra"] = "please"
assert root == boot


shoe = root.clone()
shoe.attrs["extra"] = "please2"
assert root != shoe

### Logic

#### Basic iteration of all the elements

If you just need to iterate through all the elemnts, use `iter_data`

In [ ]:
#| export
def iter_data(obj, level=0):
    """Simply yields parent and then children"""
    yield obj, level
    for child in obj.children:
        yield from iter_data(child, level=level+1)

Altough `__str__` already implements a better version of this, here's an example, using `iter_data` on how to pretty print the tree of a `Data` instance

In [ ]:
def print_data(a):
    """Prints the tree of a Data instance"""
    for obj, level in iter_data(a):
        print("   " * level, obj.name+"::"+obj.attrs['name'])
        
print_data(james)

 person::james
    person::olive
       person::andrew
       person::olive
       person::olive
    person::silva
    person::james


#### Map

To do a `map-like` sequencial processing, you can use `map_data` which will return instances of `MappedData`

In [ ]:
#| export
class MappedData(WithChildrenMixin):
    """Data structure used to return results from the `map_data` function"""
    def __init__(self, value):
        self.value = value
        super().__init__()

In [ ]:
#| export
def map_data(obj: Data, process: Callable, level=0) -> MappedData:
    """Maps over a `Data` inst returning `MappedData` instances"""
    child_results = [map_data(c, process, level=level+1) for c in obj.children]
    value = process(obj, level)
    data = MappedData(value)
    for c in child_results:
        data.add_child(c)
    return data

Here's an example showing how to process all the `Data` instances in a tree...

In [ ]:
def fun(data: Data, level: int=0):
    name = data.attrs['name']
    level_str = " " * (level+1)
    return f'{level_str}-I\'m {name}-'

result = map_data(james.clone(), fun)

...and print the result

In [ ]:
for obj, level in iter_data(result):
    print(obj.value)

 -I'm james-
  -I'm olive-
   -I'm andrew-
   -I'm olive-
   -I'm olive-
  -I'm silva-
  -I'm james-


In [ ]:
#| hide
test_eq(result.children[0].value ,"  -I'm olive-")

## Jinja2

In [ ]:
#| export
#| hide
def _get_env():
    return Environment(
        loader=BaseLoader(), 
        undefined=StrictUndefined
    )

We need a rendering function capable of renderering a `template`, with `filters` and `context variables`...

In [ ]:
#| export
#| hide
def render(
    template: str, # template in string form
    filters: Optional[dict]=None, # jinja filters
    **kwargs: Any,
) -> str:
    if not filters:
        filters = {}
    
    env = _get_env()
    env.filters.update(filters)
    
    jinja: Template = env.from_string(template)
    result: str = jinja.render(**kwargs)
        
    return result

In [ ]:
showdoc.show_doc(render)

---

### render

>      render (template:str, filters:Optional[dict]=None, **kwargs:Any)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| template | str |  | template in string form |
| filters | Optional | None | jinja filters |
| kwargs | Any |  |  |
| **Returns** | **str** |  |  |

In [ ]:
#| hide
filters = {"upper": lambda n: n.upper()}
template = 'this is a template is this is my name "{{ name | upper }}" in upper case'
kwargs = dict(name="mauro")

result = render(template, filters=filters, **kwargs)
expected = 'this is a template is this is my name "MAURO" in upper case'
test_eq(result, expected)

try:
    render("{{ name }}")
except UndefinedError:
    pass

## Frontmatter

Later one we will also use frontmatter to make our code generator more powerful. For now, we only need some helpers

In [ ]:
#| hide
#| export
class FrontMatter:
    def __init__(self, handler=None):
        if handler is None:
            handler = YAMLHandler()
        self.handler = handler

    def split(self, raw_content, *, encoding="utf-8"):
        raw_content = u(raw_content, encoding).strip()

        try:
            fm, content = self.handler.split(raw_content)
        except ValueError:
            return None, raw_content

        return fm, content

    def parse(self, raw_frontmatter, *, metadata=None):
        if metadata is None:
            metadata = {}

        try:
            raw_frontmatter = self.handler.load(raw_frontmatter)
        except ParserError as e:
            msg = dedent(f"""
            ===
            There is an error with the following yaml (front matter)
            
            ```
            {raw_frontmatter}
            ```

            ===

            """)

            print(msg)
            raise e

        if isinstance(raw_frontmatter, dict):
            metadata.update(raw_frontmatter)

        return metadata

    
    def get_content(self, template):
        frontmatter, content = self.split(template)
        return content.strip()
    
    def get_raw_frontmatter(self, template):
        resp = self.split(template)
        frontmatter, content = resp
        if frontmatter:
            return frontmatter.strip()


## Argument parsing

In [ ]:
#| export
import json

In [ ]:
#| export

def parse_arg(arg):
    try:
        v = json.loads(arg)
    except json.JSONDecodeError:
        v = arg
    return v

In [ ]:
# TODO test this
# def eval_arg(arg, locals_):
#    return eval(arg, None, locals_)

In [ ]:
#| hide
for raw, expected  in [
    # number
    ["1", 1], 
    ["1.1", 1.1], 
    
    # string
    ["santos", "santos"], 
    
    # json
    ['{"name": "santos"}', {"name": "santos"}], 
    ["[1, 2, 3]", [1, 2, 3]], 
    ['["1", "2"]', ["1", "2"]],
    
    # bool
    ['true', True],
    ['false', False],
]:
    v = parse_arg(raw)
    # print(v, type(v))
    test_eq(v, expected)
    

In [ ]:
#| export

def parse_attrs(attrs):
    for k, y in attrs.items():
        attrs[k] = parse_arg(y)
    return attrs

In [ ]:
#| hide
raw = {
    "name": "john", # string
    "age": "1", # int
    "money": "98123.45", # float
    "happy": "true", # bool
    "sad": "false", # bool
    "colors": '["red", "blue"]' # json
}
expected = {"name": "john", "age": 1, "money": 98123.45, "happy": True, "sad": False, "colors": ["red", "blue"]}
parsed = parse_attrs(raw)

test_eq(parsed, expected)

---

In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()

In [ ]:
my_dict = {"a":1, "b":2, "c":3}
{**my_dict, "c": my_dict["c"] * 2}

{'a': 1, 'b': 2, 'c': 6}